# Ejercicios para la clase práctica y tarea:

## Ejercicio 1: Traducción
Multilingual Machine Translation (MMT). En este ejercicio entrenaremos... un modelo de traducción del lenguaje natural, para los idiomas inglés y alemán.

### Datos:

https://arxiv.org/pdf/2010.11125.pdf
pagina 7 para coleccion de datasets de traduccion

### Objetivo:

que los estudiantes sean capaces de reconocer:
* la arquitectura básica de los transformer, encoder, decoder, positional ...
* cómo y para qué usar la máscara 
* cómo entrenarlos y 
* cómo utilizarlos



### Data Processing
Para el primer ejercicio utilizaremos ***torchtext***, que es la librería de pytorch que ofrece utilidades y datos para trabajar con NLP. En particular utilizaremos para este ejemplo:

*   get_tokenizer del torchtext.data.utils
*   Vocab del torchtext.vocab
*   download_from_url, extract_archive del torchtext.utils

En este primer ejemplo la vamos a usar para el procesamiento de los datos en función del modelo de traducción del lenguaje natural. 

Primero vamos a tokenizar una oración de texto sin formato, construiremos el vocabulario y convertiremos los tokens en tensores numéricos


Correr esta celda primero y luego reiniciar el entorno de ejecución. 

In [1]:
! python -m spacy download de_core_news_sm

     |████████████████████████████████| 14.9 MB 5.3 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=72ace8ffd30580f17cfe3d018bc3035083d003b2e906c63dbfa95ff98de4a930
  Stored in directory: /tmp/pip-ephem-wheel-cache-1j5z4j1r/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [1]:
import math
import torchtext
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torch import Tensor
import io
import time

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

In [2]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'

train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

train.de.gz: 100%|██████████| 637k/637k [00:00<00:00, 15.2MB/s]
train.en.gz: 100%|██████████| 569k/569k [00:00<00:00, 14.2MB/s]
val.de.gz: 100%|██████████| 24.7k/24.7k [00:00<00:00, 7.86MB/s]
val.en.gz: 100%|██████████| 21.6k/21.6k [00:00<00:00, 3.01MB/s]
test_2016_flickr.de.gz: 100%|██████████| 22.9k/22.9k [00:00<00:00, 6.06MB/s]
test_2016_flickr.en.gz: 100%|██████████| 21.1k/21.1k [00:00<00:00, 3.48MB/s]


In [3]:
from spacy.lang.en import English

de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

Esta próxima celda se demora 27 min con gpu, tenemos que buscar una mejor solución para la clase, parece que lo que más se demora es el counter.update(...).Para luego la solución es darle el vocabulario ya salvado: Leer https://github.com/bentrevett/pytorch-sentiment-analysis/issues/40

In [8]:
def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_))
  vocab = Vocab(counter)
  vocab.specials = ['<unk>', '<pad>', '<bos>', '<eos>']
  # vocab = Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
  return vocab

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

In [13]:
de_vocab.type

<bound method Module.type of Vocab()>

In [ ]:
folder = '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
torch.save(de_vocab, f'{folder}/de_vocab.pth')
torch.save(en_vocab, f'{folder}/en_vocab.pth')

In [ ]:
de_vocab = torch.load(f'{folder}/de_vocab.pth')
en_vocab = torch.load(f'{folder}/en_vocab.pth')

tiempo de ejecución: 28 min Esto tb se demora otro montón ... probablemente tenga más sentido salval estos datos que los vocabularios. Dejarle los códigos a los muchachos para que los corran por su cuenta y tengan referencia para la tarea peroen la clase solo darle el código de cargar los datos

In [ ]:
def data_process(filepaths):
  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de.rstrip("\n"))],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en.rstrip("\n"))],
                            dtype=torch.long)
    data.append((de_tensor_, en_tensor_))
  return data


train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

### DataLoader
The last torch specific feature we’ll use is the DataLoader, which is easy to use since it takes the data as its first argument. Specifically, as the docs say: DataLoader combines a dataset and a sampler, and provides an iterable over the given dataset. The DataLoader supports both map-style and iterable-style datasets with single- or multi-process loading, customizing loading order and optional automatic batching (collation) and memory pinning.

Please pay attention to collate_fn (optional) that merges a list of samples to form a mini-batch of Tensor(s). Used when using batched loading from a map-style dataset.

In [ ]:
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [ ]:


torch.save(train_iter, f'{folder}/train_iter')
torch.save(valid_iter, f'{folder}/valid_iter')
torch.save(test_iter, f'{folder}/test_iter')

In [ ]:
train_iter = torch.load(f'{folder}/train_iter')
valid_iter = torch.load(f'{folder}/valid_iter')
test_iter = torch.load(f'{folder}/test_iter')

### Transformer!
Transformer is a Seq2Seq model introduced in “Attention is all you need” paper for solving machine translation task. Transformer model consists of an encoder and decoder block each containing fixed number of layers.

Encoder processes the input sequence by propogating it, through a series of Multi-head Attention and Feed forward network layers. The output from the Encoder referred to as memory, is fed to the decoder along with target tensors. Encoder and decoder are trained in an end-to-end fashion using teacher forcing technique.

Text tokens are represented by using token embeddings. Positional encoding is added to the token embedding to introduce a notion of word order.

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

We create a subsequent word mask to stop a target word from attending to its subsequent words. We also create masks, for masking source and target padding tokens

Define model parameters and instantiate model

In [ ]:
SRC_VOCAB_SIZE = len(de_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 16

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

In [ ]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)

In [ ]:
def evaluate(model, val_iter):
  model.eval()
  losses = 0
  for idx, (src, tgt) in (enumerate(valid_iter)):
    src = src.to(device)
    tgt = tgt.to(device)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
    tgt_out = tgt[1:,:]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()
  return losses / len(val_iter)

Entrenar el modelo

de 19 a 10 min

In [ ]:
for epoch in range(1, NUM_EPOCHS+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_iter, optimizer)
  end_time = time.time()
  val_loss = evaluate(transformer, valid_iter)
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 5.316, Val loss: 4.065, Epoch time = 35.168s
Epoch: 2, Train loss: 3.727, Val loss: 3.285, Epoch time = 36.240s
Epoch: 3, Train loss: 3.131, Val loss: 2.881, Epoch time = 38.013s
Epoch: 4, Train loss: 2.741, Val loss: 2.625, Epoch time = 38.112s
Epoch: 5, Train loss: 2.454, Val loss: 2.428, Epoch time = 37.905s
Epoch: 6, Train loss: 2.223, Val loss: 2.291, Epoch time = 38.161s
Epoch: 7, Train loss: 2.030, Val loss: 2.191, Epoch time = 38.123s
Epoch: 8, Train loss: 1.866, Val loss: 2.104, Epoch time = 38.119s
Epoch: 9, Train loss: 1.724, Val loss: 2.044, Epoch time = 38.123s
Epoch: 10, Train loss: 1.600, Val loss: 1.994, Epoch time = 38.075s
Epoch: 11, Train loss: 1.488, Val loss: 1.969, Epoch time = 38.171s
Epoch: 12, Train loss: 1.390, Val loss: 1.929, Epoch time = 37.945s
Epoch: 13, Train loss: 1.299, Val loss: 1.898, Epoch time = 38.172s
Epoch: 14, Train loss: 1.219, Val loss: 1.885, Epoch time = 38.110s
Epoch: 15, Train loss: 1.141, Val loss: 1.890, Epoch time

The models trained using transformer architecture — train faster and converge to a lower validation loss compared to RNN models.

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys

In [ ]:
def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
  model.eval()
  tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
  num_tokens = len(tokens)
  src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
  return " ".join([tgt_vocab.itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [ ]:
translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu .", de_vocab, en_vocab, de_tokenizer)

' A group of people stand in front of an igloo . '

Output: A group of people stand in front of an igloo .

## Ejercicio 2: Vision transformer

import timm.models.vision_transformer para vision transformer

### Clip

paper: https://arxiv.org/pdf/2103.00020.pdf

code: https://github.com/openai/CLIP

blog: https://openai.com/blog/clip/

bueno eso sería para tener un modelo pretrained y hacer testing solo

esto sería para image to seq (descriptions)

### Deit:
https://ai.facebook.com/blog/data-efficient-image-transformers-a-promising-new-technique-for-image-classification

https://pytorch.org/tutorials/beginner/vt_tutorial.html


### Datos:

MNIST

Por el momento está fácil hacer un poco de entretenimiento con clip, y luego separarle un ejercicio para que completen con el modelo sin entrenar. Usando las funciones de la implementación de clip para transformer, cómo codificar la imagen, cómo codificar el texto, cómo hacer la clasificación.

La idea del ejercicio puede ser que ellos jueguen con un ejemplo interactivo, preentrenado como clip. Luego el foco del ejercicio estaría en que encuentren cuál sería la diferencia entre:

* la representación de imágenes como secuencias, para ser usadas en transformers
* cómo se haría la clasificación usando transformers
* cuál es la diferencia con el ejercicio anterior respecto a los objetivos: encoding, masks ...

con cuál de los dos modelos anteriores es más fácil hacer este ejercicio para nosotros

## Ejercicio 3: Sentiment Analysis

### Datos:

http://sabcorpus.linkeddata.es/ A corpus of tweets in Spanish annotated with the sentiment analysis towards brands.

http://tass.sepln.org/tass_data/download.php A corpus of texts in Spanish tagged for sentiment analysis related tasks. It is divided into several subsets created for the various tasks proposed in the different editions through the years.

Cómo sería el sistema de evaluación, hay que hacer el código completo y solo dejarle lo que tienen que completar o es dejarle el ejercicio para que ellos lo resuelvan, desde cero?

### Objetivos, repasar los objetivos de la clase práctica en un ejercicio:

* cómo se haría la clasificación usando transformers
* la arquitectura básica de los transformer, encoder, decoder, positional ...
* cómo y para qué usar la máscara 
* cómo entrenarlos y 
* cómo utilizarlos